In [1]:
# From WoFSCast
import sys, os 
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

from wofscast import icosahedral_mesh, model_utils, square_mesh
from wofscast import grid_mesh_connectivity 
from wofscast.square_mesh import _ChildVerticesBuilder, TriangularMesh, merge_meshes

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import cartopy.crs as ccrs
from glob import glob
import math 

def _get_max_and_mean_edge_distance(mesh, legacy_mesh):
    senders, receivers = square_mesh.faces_to_edges(mesh.faces, legacy_mesh)
    edge_distances = np.linalg.norm(
      mesh.vertices[senders] - mesh.vertices[receivers], axis=-1)
    return edge_distances.max(), edge_distances.mean() 


ModuleNotFoundError: No module named 'trimesh'

In [ ]:
positive_lon = True
legacy_mesh = True

# Positive longitude dataset 
year = '2021'
if positive_lon:
    #base_path = '/work/mflora/wofs-cast-data/datasets_2hr_zarr/'
    #fname = 'wrfwof_2021-05-15_020000_to_2021-05-15_041000__10min__ens_mem_09.zarr'
    
    # New dataset with ~260 longitudes
    base_path = '/work2/mflora/wofscast_datasets/dataset_10min_all_vars/'
    fname = 'wrfwof_2021-05-15_020000_to_2021-05-15_041000__10min__ens_mem_09.zarr'
    
    
# Negative longitude dataset 
else:
    base_path = '/work2/mflora/wofscast_datasets/dataset_10min_updated/'
    fname = 'wrfwof_2021-05-15_001500_to_2021-05-15_022500__10min__ens_mem_01.zarr'

    
dataset = xr.open_zarr(os.path.join(base_path, year, fname), chunks={})

In [ ]:
dataset

In [ ]:
# Specification of the multimesh.
mesh_size=5
domain_size = 150 
N = domain_size
tile_size = None #150
query_radius = 5.0 

meshes = (
        square_mesh.get_hierarchy_of_triangular_meshes(
            splits=mesh_size, domain_size=domain_size, tiling=tile_size, legacy_mesh=legacy_mesh))
    
merged_meshes = square_mesh.merge_meshes(meshes)

finest_mesh = meshes[-1]

grid_lat = dataset['lat'].values
grid_lon = dataset['lon'].values
dataset.close()

mesh_nodes_lon, mesh_nodes_lat = square_mesh.get_mesh_coords(finest_mesh, grid_lat, grid_lon)

max_dist, mean_dist = _get_max_and_mean_edge_distance(finest_mesh, legacy_mesh)

print(f'Max distance on Finest Mesh: {max_dist*3:.1f}km')
print(f'Mean distance on Finest Mesh: {mean_dist*3:.1f}km')

## NWP to MESH connections

In [ ]:
import scipy
def test_radius_query_indices(legacy_mesh, grid_size=25, mesh_size=2, query_radius=3.0, xlim=None): 
    # Test works! Does not involve latitude or longitude, so positive
    # or negative longitude plays no role.
    meshes = (
        square_mesh.get_hierarchy_of_triangular_meshes(
            splits=mesh_size, domain_size=grid_size, tiling=None, legacy_mesh=legacy_mesh))
    
    finest_mesh = meshes[-1]

    # Radius is interpreted in the 3 km grid spacing distance. 
    grid_indices, mesh_indices = square_mesh.radius_query_indices(grid_size, finest_mesh, 
                                                              radius=query_radius)
    
    # Grid and mesh positions are defined arbitrary 0-N, 0-N grid;
    # does not consider latitude and longitude 
    grid_positions = square_mesh.get_grid_positions(grid_size)
    mesh_positions = finest_mesh.vertices
    
    # Check if number of connections is correct
    total_connections = len(grid_indices)
    print(f"Total number of connections: {total_connections}")
    
    # Distance calculation to verify the number of points within the query radius
    distances = scipy.spatial.distance.cdist(grid_positions, mesh_positions, 'euclidean')
    expected_connections = np.sum(distances <= query_radius)
    print(f"Expected number of connections based on radius {query_radius}: {expected_connections}")

    # Verify that the number of connections matches expected
    assert total_connections == expected_connections, (
        f"Number of connections ({total_connections}) doesn't match expected ({expected_connections}).")
    
    # Create the plot
    fig, ax = plt.subplots(dpi=300, figsize=(6, 6))
    ax.scatter(grid_positions[:, 0], grid_positions[:, 1], c='blue', label='Grid Points')
    ax.scatter(mesh_positions[:, 0], mesh_positions[:, 1], c='red', label='Mesh Vertices')
    ax.set(title=f'Grid-Mesh Connectivity (Radius = {query_radius})')
    
    # Plot the connectivity (edges)
    for grid_idx, mesh_idx in zip(grid_indices, mesh_indices):
        ax.plot(
            [grid_positions[grid_idx, 0], mesh_positions[mesh_idx, 0]],
            [grid_positions[grid_idx, 1], mesh_positions[mesh_idx, 1]],
            'k-', alpha=0.5  # line between grid and mesh points
        )
    
    if xlim:
        ax.set_xlim([-1, xlim+1])
        ax.set_ylim([-1, xlim+1])
    
    if grid_size < 50:
        ax.legend()
    
    ax.grid(True)
    
    return ax 
    
# Run the test
test_radius_query_indices(legacy_mesh, 
                          grid_size=domain_size, 
                          mesh_size=mesh_size, 
                          query_radius=query_radius, 
                          xlim=15)  
    

## Create edge features for Grid2Mesh 

In [ ]:
def test_edge_features_grid2mesh(grid_lon, grid_lat, legacy_mesh=False):
    grid_size = 150
    mesh_size = 5
    query_radius = 5.0
    meshes = (
        square_mesh.get_hierarchy_of_triangular_meshes(
            splits=mesh_size, domain_size=grid_size, tiling=None, legacy_mesh=legacy_mesh))
    
    finest_mesh = meshes[-1]
    
    mesh_nodes_lon, mesh_nodes_lat = square_mesh.get_mesh_coords(finest_mesh, grid_lat, grid_lon)    

    # Radius is interpreted in the 3 km grid spacing distance. 
    # These are the indices of the grid and mesh positions that are connected. 
    grid_indices, mesh_indices = square_mesh.radius_query_indices(grid_size, finest_mesh, 
                                                              radius=query_radius)
    
    spatial_features_kwargs = dict(
        add_node_positions=False,
        add_node_latitude=False,
        add_node_longitude=False,
        add_relative_positions=True,
        relative_longitude_local_coordinates=True,
        relative_latitude_local_coordinates=True,
    )

    # Edges sending info from grid to mesh.
    senders = grid_indices
    receivers = mesh_indices

    print(f'{senders.shape=}, {receivers.shape=}')

    # Initialize lat and lon for the grid.
    grid_nodes_lon, grid_nodes_lat = np.meshgrid(grid_lon, grid_lat)
    grid_nodes_lon = grid_nodes_lon.reshape([-1]).astype(np.float32)
    grid_nodes_lat = grid_nodes_lat.reshape([-1]).astype(np.float32)

    # Precompute structural node and edge features according to config options.
    # Structural features are those that depend on the fixed values of the
    # latitude and longitudes of the nodes.
    (senders_node_features, receivers_node_features,
     edge_features) = model_utils.get_bipartite_graph_spatial_features(
         senders_node_lat=grid_nodes_lat,
         senders_node_lon=grid_nodes_lon,
         receivers_node_lat=mesh_nodes_lat,
         receivers_node_lon=mesh_nodes_lon,
         senders=senders,
         receivers=receivers,
         edge_normalization_factor=None,
         **spatial_features_kwargs,
     )
    
    print(f'{edge_features.shape=}')
    
    
    return edge_features
    
legacy_mesh = True 
#legacy_mesh = False

# No sensitivity to the MESH edges or connections, so insensitive to legacy_mesh

edge_features = test_edge_features_grid2mesh(grid_lon, grid_lat, legacy_mesh)   

In [ ]:
base_path = '/work/mflora/wofs-cast-data/datasets_2hr_zarr/'
fname = 'wrfwof_2021-05-15_020000_to_2021-05-15_041000__10min__ens_mem_09.zarr'
dataset_current = xr.open_zarr(os.path.join(base_path, year, fname), chunks={})

current_grid_lat, current_grid_lon = dataset_current.lat.values, dataset_current.lon.values
    
# New dataset with ~260 longitudes
base_path = '/work2/mflora/wofscast_datasets/dataset_10min_all_vars/'
fname = 'wrfwof_2021-05-15_020000_to_2021-05-15_041000__10min__ens_mem_09.zarr'
dataset_new = xr.open_zarr(os.path.join(base_path, year, fname), chunks={})

new_grid_lat, new_grid_lon = dataset_new.lat.values, dataset_new.lon.values


In [ ]:
current_edge_features = test_edge_features_grid2mesh(current_grid_lon, current_grid_lat, legacy_mesh)  

In [ ]:
new_edge_features = test_edge_features_grid2mesh(new_grid_lon, new_grid_lat, legacy_mesh)  

In [ ]:
diff = new_edge_features - current_edge_features
diff.max(), diff.min()

In [ ]:
current_edge_features[:5][1]

In [ ]:
new_edge_features[:5][1]

In [ ]:
import numpy as np
import xarray as xr

# Example: small domain with lat/lon values
senders_node_lat = np.array([33, 34, 35])  # Sender latitudes
senders_node_lon = np.array([-100, -99, -98])  # Sender longitudes in [-180, 180]
receivers_node_lat = np.array([33, 34, 35])  # Receiver latitudes
receivers_node_lon = np.array([-99.5, -98.5, -97.5])   # Receiver longitudes in [-180, 180]

# Edge sender/receiver indices
senders = np.array([0, 1, 2])
receivers = np.array([0, 1, 2])

senders_node_lon_180 = np.where(senders_node_lon < 0, senders_node_lon + 180, senders_node_lon)
receivers_node_lon_180 = np.where(receivers_node_lon < 0, receivers_node_lon + 180, receivers_node_lon)

print(f'{senders_node_lon_180=}')


# Case 1: Longitudes in the range [-180, 180]
(sender_features_180, rec_features_180, 
 edge_features_180) = model_utils.get_bipartite_graph_spatial_features(
    senders_node_lat=senders_node_lat,
    senders_node_lon=senders_node_lon_180,
    senders=senders,
    receivers_node_lat=receivers_node_lat,
    receivers_node_lon=receivers_node_lon_180,
    receivers=receivers,
    add_node_positions=True,
    add_node_latitude=True,
    add_node_longitude=True,
    add_relative_positions=True,
    relative_longitude_local_coordinates=True,
    relative_latitude_local_coordinates=True
)

# Case 2: Longitudes in the range [0, 360]
senders_node_lon_0_to_360 = (senders_node_lon) % 360
receivers_node_lon_0_to_360 = (receivers_node_lon) % 360

print(f'{senders_node_lon_0_to_360=}')


(sender_features_360, rec_features_360, 
 edge_features_360) = model_utils.get_bipartite_graph_spatial_features(
    senders_node_lat=senders_node_lat,
    senders_node_lon=senders_node_lon_0_to_360,
    senders=senders,
    receivers_node_lat=receivers_node_lat,
    receivers_node_lon=receivers_node_lon_0_to_360,
    receivers=receivers,
    add_node_positions=True,
    add_node_latitude=True,
    add_node_longitude=True,
    add_relative_positions=True,
    relative_longitude_local_coordinates=True,
    relative_latitude_local_coordinates=True
)

# Compare the edge features
print("Edge features ([0, 180] longitudes):\n", edge_features_180)
print("Edge features ([0, 360] longitudes):\n", edge_features_360)


In [ ]:
sender_features_360

In [ ]:
sender_features_180

In [ ]:
diff = np.round(edge_features_180,8) - np.round(edge_features_360,8)
print(diff.max())

## Test Mesh Edge Features

In [ ]:
def check_redundant_edges(senders, receivers):
    edges = set()
    redundant_edges = []
    
    for sender, receiver in zip(senders, receivers):
        edge = (sender, receiver)
        
        if edge in edges:
            redundant_edges.append(edge)  # Add to redundant list
        else:
            edges.add(edge)
    
    if redundant_edges:
        print("Redundant edges found:", redundant_edges)
    else:
        print("No redundant edges found.")



def test_mesh_grid(grid_lat, grid_lon, legacy_mesh):
    spatial_features_kwargs = dict(
        add_node_positions=False,
        add_node_latitude=False,
        add_node_longitude=False,
        add_relative_positions=True,
        relative_longitude_local_coordinates=True,
        relative_latitude_local_coordinates=True,
    )
    
    grid_size = 150
    mesh_size = 3

    meshes = (
        square_mesh.get_hierarchy_of_triangular_meshes(
            splits=mesh_size, domain_size=grid_size, tiling=None, legacy_mesh=legacy_mesh))

    finest_mesh = meshes[-1]
    merged_mesh = square_mesh.merge_meshes(meshes)

    # Work simply on the mesh edges.
    senders, receivers = square_mesh.faces_to_edges(merged_mesh.faces, legacy_mesh)

    print(f'After correction: {senders.shape=}, {receivers.shape=}')
    check_redundant_edges(senders, receivers)
    
    mesh_nodes_lon, mesh_nodes_lat = square_mesh.get_mesh_coords(finest_mesh, 
                                                                 grid_lat, grid_lon)
    
    node_features, edge_features = model_utils.get_graph_spatial_features(
        node_lat=mesh_nodes_lat,
        node_lon=mesh_nodes_lon,
        senders=senders,
        receivers=receivers,
        **spatial_features_kwargs,
    )
    
    # Bi-directionality check
    # Create a set of (sender, receiver) pairs
    edge_pairs = set(zip(senders, receivers))

    # Check if all edges are bi-directional
    bi_directional = True
    for sender, receiver in zip(senders, receivers):
        if (receiver, sender) not in edge_pairs:
            bi_directional = False
            print(f"Missing reverse edge for ({sender}, {receiver})")
    
    if bi_directional:
        print("All edges are bi-directional.")
    else:
        print("Some edges are not bi-directional.")

    # Create the plot
    fig, ax = plt.subplots(dpi=300, figsize=(8, 8))

    # Plot the grid and mesh nodes
    ax.scatter(mesh_nodes_lon, mesh_nodes_lat, color='red', label='Mesh Nodes', s=10)

    # Plot the connections between senders and receivers with arrows
    for sender, receiver in zip(senders, receivers):
        ax.annotate("",
                xy=(mesh_nodes_lon[receiver], mesh_nodes_lat[receiver]),
                xytext=(mesh_nodes_lon[sender], mesh_nodes_lat[sender]),
                arrowprops=dict(arrowstyle="->", color='gray', alpha=0.5))
        
#legacy_mesh = True 
legacy_mesh = False
        
test_mesh_grid(grid_lat, grid_lon, legacy_mesh)

## Test MESH2GRID Edge Features

In [ ]:
def test_mesh2grid_edges(grid_lat, grid_lon, legacy_mesh): 
    mesh2grid_edge_normalization_factor = None
    spatial_features_kwargs = dict(
        add_node_positions=False,
        add_node_latitude=False,
        add_node_longitude=False,
        add_relative_positions=True,
        relative_longitude_local_coordinates=True,
        relative_latitude_local_coordinates=True,
    )
    
    grid_size = len(grid_lat)
    mesh_size = 5

    meshes = (
        square_mesh.get_hierarchy_of_triangular_meshes(
            splits=mesh_size, domain_size=grid_size, tiling=None, legacy_mesh=legacy_mesh))

    finest_mesh = meshes[-1]
    
    # Create some edges according to how the grid nodes are contained by
    # mesh triangles.
    (grid_indices,
     mesh_indices) = square_mesh.in_mesh_triangle_indices(
         grid_size=grid_size,
         mesh=finest_mesh)
    
    
    print(f'{grid_indices.shape=}, {mesh_indices.shape=}')
    
    print(grid_indices)
    print(mesh_indices)
    
    mesh_nodes_lon, mesh_nodes_lat = square_mesh.get_mesh_coords(finest_mesh, 
                                                                 grid_lat, grid_lon)
    
    mesh_nodes_lon = np.array(mesh_nodes_lon)
    mesh_nodes_lat = np.array(mesh_nodes_lat)
    
    
    print(f'{mesh_nodes_lon.shape=}, {mesh_nodes_lat.shape=}')
    
    # Edges sending info from mesh to grid.
    senders = mesh_indices
    receivers = grid_indices
    
    (senders_node_features, receivers_node_features,
     edge_features) = model_utils.get_bipartite_graph_spatial_features(
         senders_node_lat=mesh_nodes_lat,
         senders_node_lon=mesh_nodes_lon,
         receivers_node_lat=grid_lat,
         receivers_node_lon=grid_lon,
         senders=senders,
         receivers=receivers,
         edge_normalization_factor=mesh2grid_edge_normalization_factor,
         **spatial_features_kwargs,
     )
    
legacy_mesh = False
    
test_mesh2grid_edges(grid_lat, grid_lon, legacy_mesh)    